In [8]:
using OrdinaryDiffEq
using LinearAlgebra
using DiffEqSensitivity
using Optim
using Zygote
using DelimitedFiles

In [2]:
" Reshape a vector of parameters into the variables we know. "
@views function reshapeParams(p)
    w = reshape(p[1:6889], (83, 83))
    alpha = p[6890:6972]
    eps = p[6973:7055]
    
    @assert length(eps) == 83
    @assert length(alpha) == 83
    
    return w, alpha, eps
end

" Melt the variables back into a parameter vector. "
function unshapeParams(w, alpha, eps)
    return vcat(vec(w), alpha, eps)
end

" The ODE equations we're using. "
function ODEeq(du, u, p, t)
    w, alpha, eps = reshapeParams(p)
    
    du .= eps .* (1 .+ tanh.(w * u)) .- alpha .* u
end

" Solve the ODE system. "
function solveODE(ps)
    u0 = zeros(83)
    tspan = (0.0, 10000.0)
    prob = ODEProblem(ODEeq, u0, tspan, ps)
    sol = last(solve(prob, AutoTsit5(TRBDF2()); saveat = tspan[2], reltol=1e-8, abstol=1e-8))
    return sol
end

ps = ones(9408)
solveODE(ps);

In [10]:
" Remove the effect of one gene across all others to simulate the KO experiments. "
function simKO(pIn, geneNum)
    pIn = copy(pIn) # Need to copy as we're using views
    w, alpha, eps = reshapeParams(pIn)
    
    w[:, geneNum] .= 0.0 # Think we remove a column since this is the effect of one gene across all genes
    
    pIn = unshapeParams(w, alpha, eps)
    
    return solveODE(pIn)
end

simKO(ps, 1)

83-element Array{Float64,1}:
 2.0
 2.0
 2.0
 2.0
 2.0
 2.0
 2.0
 2.0
 2.0
 2.0
 2.0
 2.0
 2.0
 ⋮
 2.0
 2.0
 2.0
 2.0
 2.0
 2.0
 2.0
 2.0
 2.0
 2.0
 2.0
 2.0

In [34]:
" Helper function to form solution matrix from solving model. "
function find_sol(pIn)
    sol = ones(83, 84)
    for i = 1:83
        sol[:, i] = simKO(pIn, i)
    end
    sol[:, 84] = solveODE(pIn)
    return sol
end

find_sol

In [35]:
" Cost function. Returns SSE between model and experimental RNAseq data. "
function cost(pIn)
    exp_data = get_data("./de/data/exp_data.csv")
    sol = ones(83, 84)
    for i = 1:83
        sol[:, i] = simKO(pIn, i)
    end
    sol[:, 84] = solveODE(pIn)
    return norm(sol .- exp_data)
end

# We can also take the gradient of the cost
grads = Zygote.gradient(cost, ps);

ErrorException: Mutating arrays is not supported

In [7]:
function g!(G, x)
    grads = Zygote.gradient(cost, x)
    G[:] .= grads[1]
end

#optimize(cost, g!, ps, LBFGS(), Optim.Options(iterations = 10, show_trace = true))

┌ Warning: `CSV.read(input; kw...)` is deprecated in favor of `DataFrame!(CSV.File(input; kw...))`
└ @ CSV /home/madisonw/.julia/packages/CSV/W9RT2/src/CSV.jl:40


ErrorException: Mutating arrays is not supported

In [9]:
" Load the experimental data matrix. "
function get_data(path_RNAseq)
    # Import RNAseq data as 83 x 84 matrix preprocessed using python
    exp = DelimitedFiles.readdlm(path_RNAseq, ',', Float64)
    exp = Matrix(exp)
    return exp
end

get_data